In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
%load_ext bigquery_magics

data_set = "testing_set"
project_name = "emerald-entity-468916-f9"

job_config = bigquery.QueryJobConfig(default_dataset = f"{project_name}.{data_set}")
magics.context.default_query_job_config = job_config

In [ ]:
%%bigquery

call phase3_prepare_chunks();

Query is running:   0%|          |

""


In [ ]:
%%bigquery

call phase4_fragments_summarization();
call phase4_summarize_fragments_summaries();

Query is running:   0%|          |

""


In [ ]:
%%bigquery

call phase5_prepare_character_ids_from_fragments();
call phase5_prepare_character_ids_initial_embeddings();

call phase5_merge_characters_duplicates();

BEGIN
    DECLARE iteration INT64 DEFAULT 0;
    REPEAT
        SET iteration = iteration + 1;
        call phase5_merge_characters_duplicates();
        delete from identifiers where importance <= iteration;
        delete from tmp_characters_id_embeddings where id not in (select id from identifiers);
    UNTIL iteration >= 7
    END REPEAT;
END;

BEGIN
  DECLARE merged INT64 DEFAULT 0;
  REPEAT
    call phase5_merge_characters_duplicates_with_return_param(merged);
  UNTIL merged = 0
  END REPEAT;
END;

call phase5_rebuild_indentifier_jsons();

Query is running:   0%|          |

""


In [ ]:
%%bigquery

CALL phase6_gather_characters_full_data_from_chunks();

CALL phase6_group_the_same_characters_data_for_merging();
CALL phase6_merge_data();

CALL phase6_copy_final_data();

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 20, "trait_type": "values", "trait_desc": 'values: "Core principles, moral compass, priorities (can include both positive and negative values)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.values as traits from characters c WHERE c.values is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET values_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 10, "trait_type": "wealth", "trait_desc": 'wealth: "Economic position, assets, property, financial struggles or abundance with information how wealth/income is obtained (inheritance, labor, trade, crime, patronage, etc.)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.wealth as traits from characters c WHERE c.wealth is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET wealth_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 7, "trait_type": "social_class", "trait_desc": 'social_class: "Economic and social standing (e.g., nobility, working class, merchant class)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.social_class as traits from characters c WHERE c.social_class is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET social_class_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Query is running:   0%|          |

""
